In [14]:
import pandas as pd
import numpy as np


In [11]:
#Open the src\data\gadm41_COD_2.json
import json
import os
# Get cwd
cwd = os.getcwd()
# cd .. to get to the parent directory
cwd = os.path.dirname(cwd)
with open(os.path.join(cwd, 'data', 'gadm41_COD_2.json')) as f:
    data = json.load(f)

In [167]:
# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'LifeExp', 'API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv'))
# Get the rows with Indicator Name = Life expectancy at birth, total (years)
df = df[df['Indicator Name'] == 'Life expectancy at birth, total (years)']
# Exclude the rows which are county aggregates
df = df[df['Country Code'] != 'COD']
df.drop(['Country Name', 'Indicator Name', 'Indicator Code', 'Unnamed: 66', '2021'], axis=1, inplace=True)

# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
c_codes = pd.read_csv(os.path.join(cwd, 'data', 'country_codes.csv'))
# Icodes = c_codes['alpha-3'].values
iso_codes = c_codes['alpha-3'].values

#Keep the rows with the iso codes in the iso_codes list
df = df[df['Country Code'].isin(iso_codes)]
df.set_index('Country Code', inplace=True)

In [168]:
#Turn into a nested dictionary
df = df.to_dict(orient='index')
# Read into a dataframe
df = pd.DataFrame.from_dict(df, orient='index')

In [169]:
# Turn into a nested dataframe
df = df.stack().reset_index()
# Rename the columns to 'country' and 'year' and 'life_exp'
df.columns = ['country', 'year', 'life_exp']

In [172]:
nested_df = df.groupby(['country', 'year']).last()

In [173]:
nested_df

life_exp
country year          
ABW     1960    64.152
        1961    64.537
        1962    64.752
        1963    65.132
        1964    65.294
...                ...
ZWE     2016    60.306
        2017    60.709
        2018    61.414
        2019    61.292
        2020    61.124

[12093 rows x 1 columns]